<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/Spark_Structured_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Spark Structured Streaming
## Introduction

In this lab, we want to understand basics of SPARK STRUCTURED STREAMING.

Just consider an infinite stream of events: 
* the stream of wikipedia updates, 
* the stream of twitter messages with an hashtag, 
* the stream of measurement from a sensor network,
* The stream can be also a combination of streams coming from many sources

Streams raises issues on Volumes and Velocity: possible to have very high volume very quickly...

You want to compute virtually anything on the stream:
* you want to translate wikipedia update from a language to another
* You want to compute the top10 of people contributing to wikipedia
* You want train a model to detect vandalism on Wikipedia
* You want to analyze tweets on your company to detect positive/Negative sentiments
* You want to detect anomaly on your sensor network to raise an alert
* use-case  are endless...

The challenge is to allow such computation with high volume and high velocity.

Main actors in Data Stream Processing are:
* Kafka, https://fr.wikipedia.org/wiki/Apache_Kafka 
* Storm, https://fr.wikipedia.org/wiki/Apache_Storm
* Flume, https://flume.apache.org/
* Flink, https://fr.wikipedia.org/wiki/Apache_Flink 
* Kinesis, 

SPark propose 2 models of STREAMING, DStream and Structured Streams. Structured is now the maintained model. Structured Streaming allows Exactly one semantics ie. one event is processed *exactly once*.

This is very important to handle fault tolerance as stream processing is distributed and many fault may happen. One key challenge for stream processing is to process all events exactly once, even if some part of processing have to be restarted due to a failure.

The key idea in Structured Streaming is to treat a live data stream as a table that is being continuously appended (even if the table is not fully materialized, processed events are discarded when consumed and replaced by partial aggregates for exmample)

![stream model](https://spark.apache.org/docs/latest/img/structured-streaming-stream-as-a-table.png)

The Figure bloew illustrates the Wordcount example. You receive words incrementally:
* Each time words is consumed, it is appended to the input table
* This triggers the activation of wordcount program that count how many times a word is seen (notice that, once aggregation is done, we can discard inputs in the input table)
* Next, results are printed to the output....

It is possible:
* to print all the the results at each activation. It is the "complete" output mode -> make senses for aggregation queries (group by/Count)
* to print only new elements appended in the *result* table -> nice for select queries (select(*) where (age>12))
* to print only elements that have been updated in the *result* table


![stream model](https://spark.apache.org/docs/latest/img/structured-streaming-example-model.png)




## Installation

Lets install code and data.
Please note that we will not use clustering -> only one master
We just want to see the programming model, not the performances...

In [12]:
#install software
# No clustering for this install !!
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 27.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=30d66e76230f17baff2e126f5a0e9f02ab0ead6cb1d101cc00772e714b779ab2
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [11]:
#get data
!wget -O data.txt "https://raw.githubusercontent.com/apache/spark/master/docs/graphx-programming-guide.md" 

# split the data in small files to simulate bag of words arrival
! split -l 100 --additional-suffix=".txt" data.txt data-
! ls -l



--2022-10-19 19:16:16--  https://raw.githubusercontent.com/apache/spark/master/docs/graphx-programming-guide.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53517 (52K) [text/plain]
Saving to: ‘data.txt’

data.txt            100%[===================>]  52.26K  --.-KB/s    in 0.01s   

2022-10-19 19:16:16 (4.64 MB/s) - ‘data.txt’ saved [53517/53517]

total 140
-rw-r--r-- 1 root root  6378 Oct 19 19:16 data-aa.txt
-rw-r--r-- 1 root root  5156 Oct 19 19:16 data-ab.txt
-rw-r--r-- 1 root root  4682 Oct 19 19:16 data-ac.txt
-rw-r--r-- 1 root root  4940 Oct 19 19:16 data-ad.txt
-rw-r--r-- 1 root root  5164 Oct 19 19:16 data-ae.txt
-rw-r--r-- 1 root root  5160 Oct 19 19:16 data-af.txt
-rw-r--r-- 1 root root  5081 Oct 19 19:16 data-ag.txt
-rw-r--r-- 1 root root  5

## programming

Lets starts writing programs...

In [13]:
# start the spark session

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

We consider the directory streaming as a source for streaming ie. each time a new file appears in this directory -> trigger word count !!


In [17]:
# create an empty streaming directory
# consider the directory "streaming" as a source of streaming
# currently it is empty...
!mkdir streaming

mkdir: cannot create directory ‘streaming’: File exists


In [18]:
# Declare the directory "Streaming" as a source
lines = spark.readStream.text("streaming")

# and write the "wordcount program"
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()


In [69]:
# stopping all active queries
# you will understand ;-)
for q in spark.streams.active:
  print("stopping",q.name)
  q.stop()
spark.sql(f"drop table wordcount")

stopping wordcount


DataFrame[]

In [70]:
# This code works... but not in the jupyter console...
#query = wordCounts \
#    .writeStream \
#    .outputMode("complete") \
#    .format("console") \
#    .start()


# So we write ouput in the "results" directory

query = wordCounts \
    .writeStream \
    .format("memory") \
    .outputMode("complete") \
    .queryName("wordcount") \
    .start()


#from time import sleep
#for x in range(3):
#    spark.sql("select * from wordcount order by count DESC").show(3)
#    spark.sql("select count(*) from wordcount").show()  
#    sleep(2)


# Once running drop one by one data-??.txt files in the streaming directory
# and check the result directory
# stop execution when happy...
#query.awaitTermination()

print(query.isActive)

True


In [71]:
spark.sql("select * from wordcount order by count DESC").show(3)
spark.sql("select count(*) from wordcount").show()
!cp data-aa.txt streaming
# and wait a little...
!sleep 5s

+----+-----+
|word|count|
+----+-----+
+----+-----+

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [72]:
spark.sql("select * from wordcount order by count DESC").show(3)
spark.sql("select count(*) from wordcount").show()
!cp data-ab.txt streaming
# and wait a little
!sleep 5s


+----+-----+
|word|count|
+----+-----+
+----+-----+

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [73]:
spark.sql("select * from wordcount order by count DESC").show(3)
spark.sql("select count(*) from wordcount").show()
!cp data-ac.txt streaming
# and wait a little
!sleep 5s

+----+-----+
|word|count|
+----+-----+
|    |   73|
| the|   15|
|   a|    9|
+----+-----+
only showing top 3 rows

+--------+
|count(1)|
+--------+
|     611|
+--------+



In [77]:
spark.sql("select * from wordcount order by count DESC").show(3)
spark.sql("select count(*) from wordcount").show()

+----+-----+
|word|count|
+----+-----+
|    |  269|
| the|   94|
| and|   47|
+----+-----+
only showing top 3 rows

+--------+
|count(1)|
+--------+
|     803|
+--------+



In [78]:
# you understood ??

# stopping all active queries
query.stop()


for q in spark.streams.active:
  print("stopping",q.name)
  q.stop()

# clear the wordcount talbe
spark.sql(f"drop table wordcount")

#clear the streaming directory
!rm streaming/*

stopping wordcount


As can be seen, Spark counts the number of edits made by each user in the past sixty seconds and emits updates once per second (the original batch duration of the `StreamingContext`).